```
!git lfs clone https://huggingface.co/Qwen/Qwen2-0.5B-Instruct
!git lfs clone https://huggingface.co/microsoft/Phi-3-mini-4k-instruct
```

In [ ]:
!pip install -q vllm

In [ ]:
import time
import uuid
import torch
import asyncio

from vllm import AsyncLLMEngine, SamplingParams
from vllm.engine.arg_utils import AsyncEngineArgs

user_prompt = "What are 3 fun things to do in SF?"

reference_models = [
    "./Qwen2-0.5B-Instruct",
    "./Phi-3-mini-4k-instruct",
]

aggregator_model = "./Qwen2-0.5B-Instruct"
aggregator_system_prompt = """You have been provided with a set of responses from various open-source models to the latest user query. Your task is to synthesize these responses into a single, high-quality response. It is crucial to critically evaluate the information provided in these responses, recognizing that some of it may be biased or incorrect. Your response should not simply replicate the given answers but should offer a refined, accurate, and comprehensive reply to the instruction. Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.

Responses from models:"""

async def generate(engine: AsyncLLMEngine, prompt: str, id: uuid.UUID) -> str:
    sampling_params = SamplingParams(temperature=0.8, max_tokens=30)
    try:
        results_generator = engine.generate(prompt, sampling_params, id)
        async for request_output in results_generator:
            pass
        return request_output.outputs[0].text
    except Exception as e:
        logging.error(f"Error generating response: {e}")
        return ""

async def main():
    engines = {}
    results = [] 

    for model in reference_models:
        engine_args = AsyncEngineArgs(
            model=model,
            dtype="half",  
            enforce_eager=True,
            gpu_memory_utilization=0.3,
            swap_space=3,
            max_model_len=256,
            kv_cache_dtype="fp8_e5m2",
            tensor_parallel_size=1,
            disable_log_requests=True,
        )
        engine = AsyncLLMEngine.from_engine_args(engine_args)
        engines[model] = engine

        result = await generate(engine, user_prompt, uuid.uuid4())
        results.append(result)
        
        print(f"Result for model {model}: {result}")
        torch.cuda.empty_cache()

    aggregator_prompt = aggregator_system_prompt + "\n" + "\n".join([f"{i+1}. {str(element)}" for i, element in enumerate(results)])
    print("Aggregator Prompt:\n", aggregator_prompt)

    aggregator_engine_args = AsyncEngineArgs(
        model=aggregator_model,
        dtype="half", 
        enforce_eager=True,
        gpu_memory_utilization=0.3,
        swap_space=3,
        max_model_len=256,
        kv_cache_dtype="fp8_e5m2",
        tensor_parallel_size=1,
        disable_log_requests=True,
    )
    aggregator_engine = AsyncLLMEngine.from_engine_args(aggregator_engine_args)
    final_result = await generate(aggregator_engine, aggregator_prompt, uuid.uuid4())

    print("Final Aggregated Result:", final_result)
    torch.cuda.empty_cache()

if __name__ == "__main__":
    asyncio.run(main())